In [ ]:
#!pip install monai
!python -c "import monai" || pip install -q "monai-weekly[gdown, nibabel, tqdm, ignite]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Import monai stuff and other necessary libraries
from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    Invertd,
    RepeatChannel,
    DivisiblePad,
    RandRotate90d
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob

print_config()

In [ ]:
#create a directory
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/tmp/tmpu42vmr33


Code below moves the data file in zip on google drive to temporary director. Note: you should keep the data in the same location\
Note: temporary files are deleted everytime you start a new session

In [ ]:
google_drive_file_path = '/content/drive/MyDrive/Colab Notebooks/data-science-bowl-2018.zip'
destination_dir = root_dir
shutil.copy(google_drive_file_path, destination_dir)



'/tmp/tmpu42vmr33/data-science-bowl-2018.zip'

In [ ]:
compressed_file = os.path.join(root_dir, "data-science-bowl-2018.zip")
data_dir = os.path.join(root_dir, "data-science-bowl-2018")

In [ ]:
compressed_file

'/tmp/tmpnirziv6r/data-science-bowl-2018.zip'

Unzip all the files returns pathname


In [ ]:
import os
import zipfile
import shutil

def extract_zip(input_zip, output_folder):
    with zipfile.ZipFile(input_zip, 'r') as zip_ref:
        zip_ref.extractall(output_folder)

def extract_nested_zips(input_folder, output_folder_base):
    extracted_folders = {}
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.zip'):
                input_zip = os.path.join(root, file)
                folder_name = os.path.splitext(file)[0]
                output_folder = os.path.join(output_folder_base, folder_name)
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)
                extract_zip(input_zip, output_folder)
                extracted_folders[folder_name] = output_folder
    return extracted_folders

def main():
    # Paths
    main_zip_path = compressed_file # Replace this with the path to your main zip folder
    output_folder_base = data_dir   # Replace this with the base path where you want to extract

    # Extract contents of main zip
    extract_zip(main_zip_path, output_folder_base)

    # Extract nested zip folders
    extracted_folders = extract_nested_zips(output_folder_base, output_folder_base)

    # Return the dictionary containing folder names and their paths
    return extracted_folders

# Call main function and store the result in a variable
extracted_folders_dict = main()

# Check if extracted_folders_dict is None
if extracted_folders_dict is not None:
    # Print the dictionary
    print("Extracted Folders:")
    for folder_name, folder_path in extracted_folders_dict.items():
        print(f"{folder_name}: {folder_path}")
else:
    print("Error: No folders extracted.")


Extracted Folders:
stage1_test: /tmp/tmpu42vmr33/data-science-bowl-2018/stage1_test
stage1_train: /tmp/tmpu42vmr33/data-science-bowl-2018/stage1_train
stage1_train_labels.csv: /tmp/tmpu42vmr33/data-science-bowl-2018/stage1_train_labels.csv


In [ ]:
# very rough code
#assign folder paths to folder names as variable
stage1_train_labels_csv = extracted_folders_dict.get("stage1_train_labels.csv")
stage1_train = extracted_folders_dict.get("stage1_train")
stage1_sample_submission_csv = extracted_folders_dict.get("stage1_sample_submission.csv")
stage2_sample_submission_final_csv = extracted_folders_dict.get("stage2_sample_submission_final.csv")
stage1_solution_csv = extracted_folders_dict.get("stage1_solution.csv")
stage2_test_final = extracted_folders_dict.get("stage2_test_final")
stage1_test = extracted_folders_dict.get("stage1_test2")


In [ ]:
TRAIN_PATH=stage1_train
TEST_PATH=stage1_test
train_ids=next(os.walk(TRAIN_PATH))[1]  # gets file names within in the directory train path
#test_ids=next(os.walk(TEST_PATH))[1]

IMG_HEIGHT=512
IMG_WIDTH=512
IMG_CHANNELS=3

In [ ]:
#!!! NOTE this part of code has been adapted from https://www.kaggle.com/code/keegil/keras-u-net-starter-lb-0-277 until the comment below

import numpy as np
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool_)


In [ ]:
from tqdm import tqdm
from skimage.io import imread,imshow
from skimage.transform import resize
print('Resizing training images and masks')
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH +'/'+ id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img  #Fill empty X_train with values from img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool_)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant',
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)

    Y_train[n] = mask

#!! adaptation over
Y_train=np.repeat(Y_train,3,axis=-1) # to have 3 channels in label too



In [ ]:
image_train_data=X_train
mask_data_train= Y_train


In [ ]:
#save arrays as image
from skimage.io import imsave
save_path_image=os.path.join(root_dir,'saved_array_images')
save_path_masks=os.path.join(root_dir,'saved_array_masks')
save_HE_image=os.path.join(root_dir,'saved_HE_images')
save_HE_masks=os.path.join(root_dir,'saved_HE_masks')
os.makedirs(save_path_image, exist_ok=True)
os.makedirs(save_path_masks, exist_ok=True)
os.makedirs(save_HE_image, exist_ok=True)
os.makedirs(save_HE_masks, exist_ok=True)
import cv2

for i, (image, mask) in enumerate(zip(image_train_data, mask_data_train)):
  image_path = os.path.join(save_path_image, f"image_{i}.png")
  mask_path = os.path.join(save_path_masks, f"mask_{i}.png")

  imsave(image_path, image)

# Save the RGB image

  imsave(mask_path,mask)





In [ ]:
from glob import glob
train_images = sorted(glob(os.path.join(save_path_image, "*.png")))
train_labels = sorted(glob(os.path.join(save_path_masks, "*.png")))


In [ ]:
## set the training and testing dictionaries. 10% is randomly locked away for testing post training.
## 30% of remaining for validation/tuning
##70 % of remaining for training

import random
data_dicts = [{"image": image_name, "label": label_name} for image_name, label_name in zip(train_images, train_labels)]
#lock 10% for testing
num_samples = int(len(data_dicts) * 0.1)
random.seed(45)
test_locked= random.sample(data_dicts, num_samples)
data_dicts = [item for item in data_dicts if item not in test_locked]


num_samples = int(len(data_dicts) * 0.3)
val_files=random.sample(data_dicts, num_samples)
train_files = [item for item in data_dicts if item not in val_files]



In [ ]:
print(f"training data: {train_files}")
print(f"validating data: {val_files}")

training data: [{'image': '/tmp/tmpu42vmr33/saved_array_images/image_0.png', 'label': '/tmp/tmpu42vmr33/saved_array_masks/mask_0.png'}, {'image': '/tmp/tmpu42vmr33/saved_array_images/image_10.png', 'label': '/tmp/tmpu42vmr33/saved_array_masks/mask_10.png'}, {'image': '/tmp/tmpu42vmr33/saved_array_images/image_104.png', 'label': '/tmp/tmpu42vmr33/saved_array_masks/mask_104.png'}, {'image': '/tmp/tmpu42vmr33/saved_array_images/image_105.png', 'label': '/tmp/tmpu42vmr33/saved_array_masks/mask_105.png'}, {'image': '/tmp/tmpu42vmr33/saved_array_images/image_109.png', 'label': '/tmp/tmpu42vmr33/saved_array_masks/mask_109.png'}, {'image': '/tmp/tmpu42vmr33/saved_array_images/image_11.png', 'label': '/tmp/tmpu42vmr33/saved_array_masks/mask_11.png'}, {'image': '/tmp/tmpu42vmr33/saved_array_images/image_110.png', 'label': '/tmp/tmpu42vmr33/saved_array_masks/mask_110.png'}, {'image': '/tmp/tmpu42vmr33/saved_array_images/image_111.png', 'label': '/tmp/tmpu42vmr33/saved_array_masks/mask_111.png'}, 

In [ ]:
set_determinism(seed=833877392)

In [ ]:
#Adapted from Monai tutorial by our TA and https://github.com/Project-MONAI/tutorials/blob/main/2d_segmentation/torch/unet_training_dict.py

train_transforms = Compose( #takes a list
    [
        # Load image and label data
        LoadImaged(keys=["image", "label"]),
        # Ensure channel is the first dimension (pytorch requirement) channel = feature / kernel
        EnsureChannelFirstd(keys=["image", "label"]),
        # Scale intensity values of the image within the specified range
        ScaleIntensityRanged(
            keys=["image","label"],
            a_min=0,
            a_max=255,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        RandRotate90d(keys=["image", "label"], prob=0.5, spatial_axes=[0, 1]), #rotate randomly
        # Randomly crop regions with positive and negative labels to create training samples
        RandCropByPosNegLabeld(
            keys=["image", "label"],
            label_key="label",
            spatial_size=(96, 96),
            pos=1,
            neg=1,
            num_samples=4,  # we'll get 4 random 96,96,96 image from the full 512x512 image
            image_key="image",
            image_threshold=0,
        ),

)

val_transforms = Compose(
    [
        # Load image and label data
        LoadImaged(keys=["image", "label"]),
        # Ensure channel is the first dimension
        EnsureChannelFirstd(keys=["image", "label"]),
        # Scale intensity values of the image within the specified range
        ScaleIntensityRanged(
            keys=["image"],
            a_min=0,
            a_max=255,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),


    ]
)


In [ ]:
check_ds = Dataset(data=val_files, transform=val_transforms)

# Create a DataLoader with a batch size of 1 for visualization purposes
check_loader = DataLoader(check_ds, batch_size=1)

# Retrieve the first batch of data from the DataLoader
check_data = first(check_loader)

# Extract the image and label from the data
image, label = (check_data["image"][0][0], check_data["label"][0][0])

# Print the shapes of the image and label
print(f"image shape: {image.shape}, label shape: {label.shape}")

# Plot a slice of the images and labels (slice along the third dimension at index 80)
plt.figure("check", (12, 6))
plt.subplot(1, 2, 1)
plt.title("image")
plt.imshow(image, cmap="gray")
plt.subplot(1, 2, 2)
plt.title("label")
plt.imshow(label,cmap="gray")
plt.show()

In [ ]:
train_ds = CacheDataset(data=train_files, transform=train_transforms, cache_rate=1.0, num_workers=4)

# Alternatively, for experimentation with the regular Dataset, uncomment the line below:
# train_ds = Dataset(data=train_files, transform=train_transforms)

# Use DataLoader to load training data in batches of size 2
# Shuffle the data for randomization and apply RandCropByPosNegLabeld for data augmentation
# The resulting batch size will be 2 x 4, generating 2 batches of 4 images each for network training
#num-workers shouldnt be too high, sometimes it crashes.
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=4)

# Create a CacheDataset for validation with accelerated caching
# Similar to training, CacheRate is set to 1.0, caching the entire dataset for optimal performance
# NumWorkers enables multi-threading during caching for faster data loading
val_ds = CacheDataset(data=val_files, transform=val_transforms, cache_rate=1.0, num_workers=4)

# Alternatively, for experimentation with the regular Dataset, uncomment the line below:
# val_ds = Dataset(data=val_files, transform=val_transforms)

# Use DataLoader to load validation data in batches of size 1
# NumWorkers enables multi-threading during data loading for faster processing
val_loader = DataLoader(val_ds, batch_size=1, num_workers=4)



Loading dataset: 100%|██████████| 423/423 [00:16<00:00, 25.14it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Loading dataset: 100%|██████████| 180/180 [00:08<00:00, 20.38it/s]


In [ ]:
#device = torch.device("cuda:0")
device = torch.device("cpu")
for batch_data in train_loader:  # move this do gpu
    inputs, labels = (
        batch_data["image"].to(device),
        batch_data["label"].to(device),
    )
    print(f"Input image shape: {inputs.shape}") # shouldn B x C X H X W X D
    print(f"Input label shape: {labels.shape}")
    break

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Input image shape: torch.Size([8, 3, 96, 96])
Input label shape: torch.Size([8, 3, 96, 96])


In [ ]:
model = UNet(
    spatial_dims=2, #3D data
    in_channels=3,
    out_channels=3, #foreground and background
    channels=(4,8,16,32,64), #different convolution kernel size - different features by each kernel /filter. This is a hyperparameter.  try and compare bunch of different ones to determine these numbers
    strides=(2,2,2,2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device) #to gpu
print(model)

UNet(
  (model): Sequential(
    (0): ResidualUnit(
      (conv): Sequential(
        (unit0): Convolution(
          (conv): Conv2d(3, 4, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (adn): ADN(
            (N): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
        (unit1): Convolution(
          (conv): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (adn): ADN(
            (N): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
      )
      (residual): Conv2d(3, 4, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): SkipConnection(
      (submodule): Sequential(
        (0): ResidualUnit(
          (conv): Sequential(
            (unit0): Convolution(

In [ ]:
#loss_function = DiceLoss(to_onehot_y=False, softmax=True)  #softmax turns output to probability
# apply forward
#outputs = model(inputs)
#print(f"outputs outputs shape: {outputs.shape}")
#loss = loss_function(outputs, labels)
#print(f"train_loss: {loss.item():.4f}")

In [ ]:
loss_function = DiceLoss(to_onehot_y=False, softmax=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-2)  # 1e-4 is a hyperparameter - learning rate.
# AdamW is an alternative optimizer. # Or gradient descent
dice_metric = DiceMetric(include_background=False, reduction="mean")

In [ ]:
max_epochs = 25

val_interval = 2 # do 2 epoch training then 1 validation

# Initialize variables for tracking best metric and associated epoch
best_metric = -1
best_metric_epoch = -1

# Lists to store epoch loss values and metric values
epoch_loss_values = []
metric_values = []

# Define post-processing transforms for predictions and labels
#post_pred = Compose([AsDiscrete(argmax=True, to_onehot=3)])
post_pred = Compose([AsDiscrete(argmax=True)])
#post_label = Compose([AsDiscrete(to_onehot=1)])  # as discrete makes it a binary
post_label = Compose([AsDiscrete()])
# Iterate over epochs
for epoch in range(max_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")

    # Set model to training mode
    model.train()  # dropout would work in this mode.
    epoch_loss = 0
    step = 0

    # Iterate over batches in the training loader
    for batch_data in train_loader:
        step += 1
        # Move inputs and labels to GPU
        inputs, labels = (
            batch_data["image"].to(device),
            batch_data["label"].to(device),
        )
        # Zero the gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(inputs)
        # Compute loss
        loss = loss_function(outputs, labels)
        # Backward pass
        loss.backward()
        # Update weights
        optimizer.step()
        # Update epoch loss
        epoch_loss += loss.item()
        print(f"{step}/{len(train_ds) // train_loader.batch_size}, "
              f"train_loss: {loss.item():.4f}")

    # Calculate average epoch loss
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    # Perform validation at specified intervals
    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            for val_data in val_loader:
                val_inputs, val_labels = (
                    val_data["image"].to(device),
                    val_data["label"].to(device),
                )
                # Define ROI size and sliding window batch size
                roi_size = (96,96)
                sw_batch_size = 2
                # Perform sliding window inference
                val_outputs = sliding_window_inference(val_inputs, roi_size, sw_batch_size, model)
                val_outputs = [post_pred(i) for i in decollate_batch(val_outputs)]
                val_labels = [post_label(i) for i in decollate_batch(val_labels)]  # comment out
                # Compute dice metric for current iteration
                dice_metric(y_pred=val_outputs, y=val_labels)  # now we pass on to the metric instead of loss function

            # Aggregate the final mean dice result
            metric = dice_metric.aggregate().item()
            # Reset the status for the next validation round
            dice_metric.reset()

            # Append metric value to list
            metric_values.append(metric)

            # Check if current metric is better than best metric
            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                # Save the model with the best metric
                torch.save(model.state_dict(), os.path.join(root_dir, "best_metric_model.pth"))  # saves weights of the model
                print("saved new best metric model")

            # Print current and best metric values
            print(
                f"current epoch: {epoch + 1} current mean dice: {metric:.4f}"
                f"\nbest mean dice: {best_metric:.4f} "
                f"at epoch: {best_metric_epoch}"
            )  # syntax error

In [ ]:
# Define plot parameters
plt.figure(figsize=(14, 6))

# Plot Epoch Average Loss
plt.subplot(1, 2, 1)
plt.title("Epoch Average Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
x_loss = [i + 1 for i in range(len(epoch_loss_values))]
y_loss = epoch_loss_values
plt.plot(x_loss, y_loss, color='blue', label='Training Loss')
plt.legend()

# Plot Validation Mean Dice
plt.subplot(1, 2, 2)
plt.title("Validation Mean Dice")
plt.xlabel("Epoch")
plt.ylabel("Mean Dice")
x_dice = [val_interval * (i + 1) for i in range(len(metric_values))]
y_dice = metric_values
plt.plot(x_dice, y_dice, color='green', label='Validation Dice')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Load the best saved model parameters
model.load_state_dict(torch.load(os.path.join(root_dir, "best_metric_model.pth")))

# Set the model to evaluation mode
model.eval()

# Perform inference on validation data using the best model
with torch.no_grad():
    for i, val_data in enumerate(val_loader):
        roi_size = (512,512)
        sw_batch_size = 2
        val_outputs = sliding_window_inference(val_data["image"].to(device),
                                               roi_size, sw_batch_size, model)

        # Plot the slices
        plt.figure("check", (18, 6))

        # Plot original image
        plt.subplot(1, 3, 1)
        plt.title(f"image {i}")
        plt.imshow(val_data["image"][0, 0, :, :], cmap="gray")

        # Plot ground truth label
        plt.subplot(1, 3, 2)
        plt.title(f"label {i}")
        plt.imshow(val_data["label"][0, 0, :, :])

        # Plot model output
        plt.subplot(1, 3, 3)
        plt.title(f"output {i}")
        plt.imshow(torch.argmax(val_outputs, dim=1).detach().cpu()[0,:,:])
        plt.show()

        # Break loop after 3 iterations for visualization
        if i == 5:
            break

Code below creates a dictionary where each folder is a key and contains the image and the masks that belong to that image

In [ ]:
model_dir="/content/drive/MyDrive/Colab Notebooks/data-science-bowl-2018/model_output_6/"   #saving directory

In [ ]:
import pandas as pd

# Create a sample DataFrame
output_data = {'Dice Loss': y_loss, 'Epocs': x_loss}
df = pd.DataFrame(output_data)

# Concatenate 'col1' and 'col2', split the resulting string, and store it in 'result'
#df['result'] = df['col1'].str.cat(df['col2'], sep=" ").str.split()

# Save the DataFrame to a CSV file
output_directory = model_dir  # Specify your desired output directory
output_filename = 'model_output_training_dice_loss, .csv'
df.to_csv(output_directory + output_filename, sep=',', index=False)

print(f"DataFrame saved to {output_directory}{output_filename}")

output_data = {'Mean dice': y_dice, 'Epocs': x_dice}
df = pd.DataFrame(output_data)

# Concatenate 'col1' and 'col2', split the resulting string, and store it in 'result'
#df['result'] = df['col1'].str.cat(df['col2'], sep=" ").str.split()

# Save the DataFrame to a CSV file
output_directory = model_dir  # Specify your desired output directory
output_filename = 'model_output_training_dice_mean, .csv'
df.to_csv(output_directory + output_filename, sep=',', index=False)

print(f"DataFrame saved to {output_directory}{output_filename}")



DataFrame saved to /content/drive/MyDrive/Colab Notebooks/data-science-bowl-2018/model_output_6/model_output_training_dice_loss, .csv
DataFrame saved to /content/drive/MyDrive/Colab Notebooks/data-science-bowl-2018/model_output_6/model_output_training_dice_mean, .csv


In [ ]:
torch.save(model.state_dict(), os.path.join(model_dir, "best_metric_model.pth"))

TEST

In [ ]:
# Define the list of test images
# originally we do this on test set, but here for this dataset we don't have labels for test set because it's a challenge dataset
# so here the mean dice s
# Create test data dictionary

test_data = test_locked

# Define original test transforms
test_org_transforms = val_transforms

# Create original test dataset
test_org_ds = Dataset(data=test_data, transform=test_org_transforms)

# Create data loader for original test dataset
test_org_loader = DataLoader(test_org_ds, batch_size=1, num_workers=4)

# Define post-processing transforms
post_pred = Compose([AsDiscrete(argmax=True)])
#post_label = Compose([AsDiscrete(to_onehot=0)])
post_label = Compose([AsDiscrete()])
# Load the best saved model parameters
model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/data-science-bowl-2018/model_output_6/best_metric_model.pth"))
model.eval()

# Evaluate model performance on test set
count=0
with torch.no_grad():
    for test_data in test_org_loader:
        test_inputs = test_data["image"].to(device)
        test_labels = test_data["label"].to(device)
        roi_size = (96,96)
        sw_batch_size = 4
        # Perform sliding window inference
        test_outputs1 = sliding_window_inference(test_inputs, roi_size, sw_batch_size, model)
        test_outputs2 = [post_pred(i) for i in decollate_batch(test_outputs1)]
        test_labels = [post_label(i) for i in decollate_batch(test_labels)]
        # Compute metric for current iteration
        dice_metric(y_pred=test_outputs2, y=test_labels)

     # Plot the slices   #### uncomment if you want to see an image
        #plt.figure("check", (18, 6))

        # Plot original image
        #plt.subplot(1, 3, 1)
        #plt.title(f"image {i}")
        #plt.imshow(test_data["image"][0, 0, :, :], cmap="gray")

        # Plot ground truth label
       # plt.subplot(1, 3, 2)
        #plt.title(f"label {i}")
        #plt.imshow(test_data["label"][0, 0, :, :])

        # Plot model output
        #plt.subplot(1, 3, 3)
        #plt.title(f"output {i}")
        #plt.imshow(torch.argmax(test_outputs1, dim=1).detach().cpu()[0,:,:])
        #plt.imshow()
        #plt.show(test_outputs1[0,0,:,:])

        # Break loop after 3 iterations for visualization
        #if i == 5:
            #break
        count=count+1
    # Aggregate the final mean dice result
    metric_org = dice_metric.aggregate().item()
    # Reset the status for the next validation round
    dice_metric.reset()
    count

# Print the metric on the test set
print("Metric on Test Set: ", metric_org) # as we used validation data now should be the same as best score
print("Tested with", count,"images")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Metric on Test Set:  0.7729658484458923
Tested with 67 images
